<a href="https://colab.research.google.com/github/sohmxdd/LuminaScope-Lite/blob/main/notebooks/LuminaScope_Lite_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
!git clone https://github.com/sohmxdd/LuminaScope-Lite.git
%cd LuminaScope-Lite

Cloning into 'LuminaScope-Lite'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
Receiving objects: 100% (21/21), 9.85 KiB | 9.85 MiB/s, done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Resolving deltas: 100% (7/7), done.
/content/LuminaScope-Lite/Placeholder /LuminaScope-Lite/LuminaScope-Lite/LuminaScope-Lite/LuminaScope-Lite


In [69]:
!pip install tensorflow numpy pandas matplotlib scikit-learn

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

In [71]:
mkdir /content/LuminaScope-Lite/dataset

mkdir: cannot create directory ‘/content/LuminaScope-Lite/dataset’: File exists


In [72]:
ls dataset/

ls: cannot access 'dataset/': No such file or directory


In [73]:
df = pd.read_csv("/content/LuminaScope-Lite/dataset/cumulative.csv")
df.head()


,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [74]:
df.shape

(9564, 50)

In [75]:
df = df[['koi_period', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_impact', 'koi_teq', 'koi_disposition']]
df.dropna(inplace=True)
df.head()


,koi_period,koi_duration,koi_depth,koi_prad,koi_impact,koi_teq,koi_disposition
0,9.488036,2.95750,615.8,2.26,0.146,793.0,CONFIRMED
1,54.418383,4.50700,874.8,2.83,0.586,443.0,CONFIRMED
2,19.899140,1.78220,10829.0,14.60,0.969,638.0,FALSE POSITIVE
3,1.736952,2.40641,8079.2,33.46,1.276,1395.0,FALSE POSITIVE
4,2.525592,1.65450,603.3,2.75,0.701,1406.0,CONFIRMED


In [76]:
df['label'] = df['koi_disposition'].apply(lambda x: 1 if x == 'CONFIRMED' else 0)

In [77]:
X = df.drop(['koi_disposition', 'label'], axis=1)
y = df['label']

In [78]:
# Features & labels
X = df.drop(['koi_disposition', 'label'], axis=1)
y = df['label']

# Normalize (scale between 0 and 1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Training set:", X_train.shape)
print("Testing set:", X_test.shape)

Training set: (7360, 6)
Testing set: (1841, 6)


In [79]:
model = Sequential([
    Dense(128, activation='leaky_relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='leaky_relu'),
    Dense(32, activation='leaky_relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_26 (Dense)                │ (None, 128)            │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,265 (44.00 KB)

 Trainable params: 11,265 (44.00 KB)

 Non-trainable params: 0 (0.00 B)

In [80]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=15,
    batch_size=32,
    verbose=1
)

Epoch 1/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.7362 - loss: 0.5804 - val_accuracy: 0.7473 - val_loss: 0.5039
Epoch 2/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7398 - loss: 0.4980 - val_accuracy: 0.7357 - val_loss: 0.4861
Epoch 3/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7570 - loss: 0.4662 - val_accuracy: 0.7446 - val_loss: 0.4751
Epoch 4/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7531 - loss: 0.4699 - val_accuracy: 0.7473 - val_loss: 0.4644
Epoch 5/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7431 - loss: 0.4708 - val_accuracy: 0.7452 - val_loss: 0.4642
Epoch 6/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7467 - loss: 0.4567 - val_accuracy: 0.7473 - val_loss: 0.4545
Epoch 7/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7448 - loss: 0.4559 - val_accuracy: 0.7473 - val_loss: 0.4515
Epoch 8/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7432 - loss: 0.4530 - val_accuracy: 0

In [82]:
model.save("luminascope_model.h5")

In [83]:
from google.colab import files
files.download("luminascope_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### ✅ LuminaScope Lite Final Summary
- Final Model: v2 (Deeper Neural Network)
- Test Accuracy: ~75%
- Dataset: NASA Kepler Cumulative
- Next Phase: Streamlit dashboard + LuminaScope Pro (Raw Light Curve Integration)